## Analyzing CIA Factbook Data Using SQL
In this project, I'll use SQL to explore and analyze data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:
- population - The population as of 2015.
- population_growth - The annual population growth rate, as a percentage.
- area - The total land and water area.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [3]:
%%sql
-- Get all tables in the database
SELECT * 
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We observe there are two tables in the factbook database, namely ```sqlite_sequence```, ```facts```

Next we write a query to return first 5 rows of the facts table

In [4]:
%%sql
SELECT * 
FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:
- name - The name of the country.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.

Let's start by calculating some summary statistics and look for any outlier countries.

In [5]:
%%sql
--Minimum/Maximum Population and Population_growth

SELECT  MIN(population) as min_population,
        MAX(population) as max_population,
        MIN(population_growth) as min_population_growth, 
        MAX(population_growth) as max_population_growth
FROM facts;    

Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

In [6]:
%%sql
--Least Populated Country

SELECT *
FROM facts
WHERE population = (SELECT MIN(population) 
                    FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [7]:
%%sql
--Most Populated Country

SELECT *
FROM facts
WHERE population == (SELECT MAX(population) 
                    FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook [page for Antarctica:](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html)

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

In [8]:
%%sql
--Minimum/Maximum Population and Population_growth

SELECT  MIN(population) as min_population,
        MAX(population) as max_population,
        MIN(population_growth) as min_population_growth, 
        MAX(population_growth) as max_population_growth
FROM facts
WHERE name != 'World';    

Done.


min_population,max_population,min_population_growth,max_population_growth
0,1367485388,0.0,4.02


In [9]:
%%sql
--Average Population and Area

SELECT  AVG(population) as avg_population,
        AVG(area) as avg_area
FROM facts;    

Done.


avg_population,avg_area
62094928.32231405,555093.546184739


Now, we find countries that are densely populated. To do this, we'll identify countries that have:
- Above average values for population.
- Below average values for area.

In [10]:
%%sql
SELECT * 
FROM facts
WHERE population > (SELECT  AVG(population) as avg_population
                    FROM facts) 
        AND area < (SELECT AVG(area) as avg_area
                                            FROM facts) ; 

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


In [11]:
%%sql
--Most Populated Country

SELECT *
FROM facts
WHERE population == (SELECT MAX(population) 
                    FROM facts 
                    WHERE name != 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


In [12]:
%%sql
--Fastest Growing Country

SELECT *
FROM facts
WHERE population_growth == (SELECT MAX(population_growth) 
                    FROM facts 
                    WHERE name != 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


In [13]:
%%sql
--Countries that have more water than land

SELECT name, area, area_land, area_water, 
        CAST(area_water as float)/area *100 as water_land_ratio
FROM facts
ORDER BY water_land_ratio DESC;

Done.


name,area,area_land,area_water,water_land_ratio
British Indian Ocean Territory,54400,60,54340,99.88970588235294
Virgin Islands,1910,346,1564,81.8848167539267
Puerto Rico,13791,8870,4921,35.6826916104706
"Bahamas, The",13880,10010,3870,27.881844380403457
Guinea-Bissau,36125,28120,8005,22.15916955017301
Malawi,118484,94080,24404,20.596873839505754
Netherlands,41543,33893,7650,18.41465469513516
Uganda,241038,197100,43938,18.228661040997686
Eritrea,117600,101000,16600,14.1156462585034
Liberia,111369,96320,15049,13.512736937567905


In [16]:
%%sql
--Fastest Growing Countries

SELECT name, population, 
        population_growth 
FROM facts
ORDER BY population_growth DESC
LIMIT 10;

Done.


name,population,population_growth
South Sudan,12042910,4.02
Malawi,17964697,3.32
Burundi,10742276,3.28
Niger,18045729,3.25
Uganda,37101745,3.24
Qatar,2194817,3.07
Burkina Faso,18931686,3.03
Mali,16955536,2.98
Cook Islands,9838,2.95
Iraq,37056169,2.93


In [30]:
%%sql
--Countries adding the most people next year, 2016

SELECT name, population, 
        CAST(population_growth * population as Float)/100 as population_added 
FROM facts
WHERE name != 'World'
ORDER BY population_added DESC
LIMIT 14;

Done.


name,population,population_added
India,1251695584,15270686.1248
China,1367485388,6153684.246
Nigeria,181562056,4448270.372
Pakistan,199085847,2906653.3662
Ethiopia,99465819,2874562.1691
Bangladesh,168957745,2703323.92
United States,321368864,2506677.1392
Indonesia,255993674,2355141.8008000003
"Congo, Democratic Republic of the",79375136,1944690.8320000002
Philippines,100998376,1626073.8536


In [19]:
%%sql
--Countries that have higher death rate than birth rate

SELECT name, birth_rate, death_rate
FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate DESC;

Done.


name,birth_rate,death_rate
Ukraine,10.72,14.46
Bulgaria,8.92,14.44
Latvia,10.0,14.31
Lithuania,10.1,14.27
Russia,11.6,13.69
Serbia,9.08,13.66
Belarus,10.7,13.36
Hungary,9.16,12.73
Moldova,12.0,12.59
Estonia,10.51,12.4


In [22]:
%%sql
--Countries that have higher birth rate than death rate

SELECT name, birth_rate, death_rate
FROM facts
WHERE death_rate < birth_rate
ORDER BY death_rate DESC
LIMIT 15;

Done.


name,birth_rate,death_rate
Lesotho,25.47,14.89
Guinea-Bissau,33.38,14.33
Chad,36.6,14.28
Namibia,19.8,13.91
Afghanistan,38.57,13.89
Central African Republic,35.08,13.8
Somalia,40.45,13.62
Swaziland,24.67,13.56
Botswana,20.96,13.39
Gabon,34.49,13.12


In [31]:
%%sql
--Most densly populated countries

SELECT name, population, area, 
        CAST(population as float)/area *100 as population_density
FROM facts
ORDER BY population_density DESC
LIMIT 13;

Done.


name,population,area,population_density
Macau,592731,28,2116896.4285714286
Monaco,30535,2,1526750.0
Singapore,5674472,697,814127.9770444763
Hong Kong,7141106,1108,644504.1516245487
Gaza Strip,1869055,360,519181.94444444444
Gibraltar,29258,6,487633.3333333333
Bahrain,1346613,760,177185.92105263157
Maldives,393253,298,131964.09395973154
Malta,413965,316,131001.58227848102
Bermuda,70196,54,129992.59259259258
